In [300]:
import os
import pandas as pd
import re
import numpy as np

In [317]:
income_path = '../data/income/'
processed_path = '../data/processed/'

In [318]:
income = pd.read_csv(income_path + 'income.csv')
states_name = pd.read_csv(income_path + 'states_name.csv')
population = pd.read_csv(income_path + 'population.csv')
time_series_mobility = pd.read_csv(processed_path + 'time_series_mobility.csv')

In [319]:
income_split = np.split(income, income[income.isnull().all(1)].index)[1:]
income_clean = pd.DataFrame()
for state in income_split:
    state_name = state['County'][1:2].values
    state['state'] = state_name[0]
    if state_name == 'District of Columbia':
        income_clean = income_clean.append(state[1:2])
    else:
        
        income_clean = income_clean.append(state[2:])

In [320]:
income_clean['state'] = income_clean['state'].str.lower()
income_clean['county'] = income_clean['County'].str.lower()
income_clean['2018'] = income_clean['2,018']
income_clean = income_clean.drop(['County','2,018'], axis=1)
income_clean = income_clean[['county','state','2018']]

In [321]:
population = population[1:]
population = population.drop(index_list)

In [322]:
# find index of 
index_list = []
for state in states.unique():
    index_list.append(states[(states == state)].index[0])

In [323]:
population['Area_Name'] = population['Area_Name'].map(lambda x: re.sub(' County| City','', x)).str.lower()

In [324]:
population

,FIPS,State,Area_Name,POP_ESTIMATE_2018
2,1001,AL,autauga,"55,601"
3,1003,AL,baldwin,"218,022"
4,1005,AL,barbour,"24,881"
5,1007,AL,bibb,"22,400"
6,1009,AL,blount,"57,840"
...,...,...,...,...
3189,56037,WY,sweetwater,"43,051"
3190,56039,WY,teton,"23,081"
3191,56041,WY,uinta,"20,299"
3192,56043,WY,washakie,"7,885"


In [325]:
dictionary = {short:long for (short,long) in zip(states_name['Code'], states_name['State'])}
population['State'] = population['State'].map(dictionary).str.lower()

In [335]:
population = population[['State', 'Area_Name', 'POP_ESTIMATE_2018']]
population.columns = ['state','county', 'pop_2018']

In [333]:
# merge income
income_combined = pd.merge(time_series_mobility, income_clean, on=['state','county'])

In [340]:
time_series_all  = pd.merge(income_combined, population, on=['state','county'])

In [343]:
time_series_all.to_csv(processed_path+'time_series_all.csv')